In [1]:
%display latex
from delierium import *
from pprint import pprint 
from collections.abc import Iterable
import functools
from operator import mul, sub
from IPython.core.debugger import set_trace

In [2]:
def Lfunc(e):
    return e

In [3]:
def Lcml(e):
    '''Least common left multiply, p.23'''
    return 0

In [4]:
def Lcm(e):
    return 0

In [5]:
def FindIntegrableConditions(S, context):
    return S

In [6]:
def ReduceIntegrabilityConditions(S, context):
    return S

In [7]:

var ("x y")
z = function("z")(x,y)
w = function("w")(x,y)
# ctx: items are in descending order ?
ctx = Context((w,z), (x,y), Mgrlex)
f1 =diff(w, y) + x*diff(z,y)/(2*y*(x**2+y)) - w/y
f1.show()

-w(x, y)/y + 1/2*x*diff(z(x, y), y)/((x^2 + y)*y) + diff(w(x, y), y)

In [8]:
f2=diff(z,x,y) + y*diff(w,y)/x + 2*y*diff(z, x)/x
f2.show()

y*diff(w(x, y), y)/x + 2*y*diff(z(x, y), x)/x + diff(z(x, y), x, y)

In [9]:
f3 = diff(w, x,y) - 2*x*diff(z, x,2)/y - x*diff(w,x)/y**2
f3.show()

-2*x*diff(z(x, y), x, x)/y - x*diff(w(x, y), x)/y^2 + diff(w(x, y), x, y)

In [10]:
f4 = diff(w, x,y) + diff(z, x,y) + diff(w, y)/(2*y) - diff(w,x)/y + x* diff(z, y)/y - w/(2*y**2)
f4.show()

x*diff(z(x, y), y)/y - diff(w(x, y), x)/y + 1/2*diff(w(x, y), y)/y - 1/2*w(x, y)/y^2 + diff(w(x, y), x, y) + diff(z(x, y), x, y)

In [11]:
f5 =diff(w,y,y) + diff(z,x,y) - diff(w, y)/y + w/(y**2)
f5.show()

-diff(w(x, y), y)/y + w(x, y)/y^2 + diff(w(x, y), y, y) + diff(z(x, y), x, y)

In [12]:
#f2a=f2-(y*f1/x);f2a.expand().show()

In [13]:
f1dp=Differential_Polynomial(f1, context=ctx)
f2dp=Differential_Polynomial(f2, context=ctx)
f3dp=Differential_Polynomial(f3, context=ctx)
f4dp=Differential_Polynomial(f4, context=ctx)
f5dp=Differential_Polynomial(f5, context=ctx)

In [14]:
class Janet_Basis:
    def __init__ (self, S, context):
        """List of homogenous PDE's + order context"""
        self.S = S
        if not isinstance(self.S, Iterable):
            # bad criterion
            self.S = [self.S]
        self.S = Reorder([Differential_Polynomial(s, context) for s in self.S], context, ascending = True)
        self.S = Autoreduce (self.S, context)
        #self.S = CompleteSystem(self.S, context)
        #self.S = FindIntegrableConditions(self.S, context)
        #self.S = ReduceIntegrabilityConditions(self.S, context)
        #if all ([_ == 0 for _ in c]) or not c:
        #    return S
        #S += c
        self.S = Reorder (self.S, context)
    def rank(self):
        return 0
    def order(self):
        return 0

In [15]:
%load_ext snakeviz

In [16]:
%%snakeviz
%load_ext line_profiler
#%lprun -f DTerm.__init__ 
checkS=Janet_Basis([f3,f1,f5, f2,f4], ctx)

 
*** Profile stats marshalled to file '/tmp/tmpc7_cpumi'. 
Embedding SnakeViz in this document...


In [17]:
for _ in checkS.S:
    _.show()

diff(w(x, y), x)

-w(x, y)/y + diff(w(x, y), y)

1/2*w(x, y)/y + diff(z(x, y), x)

diff(z(x, y), y)

In [18]:
MatrixOrder.higher.cache_info()

CacheInfo(hits=0, misses=52, maxsize=None, currsize=52)